#### In this task we will work on a dataset with pandas

In [1]:
# modules we'll use
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# read in 2 datasets provided
nfl_data = pd.read_csv("NFL Play by Play 2009-2016 (v3).csv", low_memory=False)
pd.set_option('display.max_columns', None) # What does this option do? Answer below in a comment

Below is my answer what `pd.set_option` function do.

In [2]:
print(pd.set_option.__doc__ )


set_option(pat, value)

Sets the value of the specified option.

Available options:

- compute.[use_bottleneck, use_numba, use_numexpr]
- display.[chop_threshold, colheader_justify, column_space, date_dayfirst,
  date_yearfirst, encoding, expand_frame_repr, float_format]
- display.html.[border, table_schema, use_mathjax]
- display.[large_repr]
- display.latex.[escape, longtable, multicolumn, multicolumn_format, multirow,
  repr]
- display.[max_categories, max_columns, max_colwidth, max_info_columns,
  max_info_rows, max_rows, max_seq_items, memory_usage, min_rows, multi_sparse,
  notebook_repr_html, pprint_nest_depth, precision, show_dimensions]
- display.unicode.[ambiguous_as_wide, east_asian_width]
- display.[width]
- io.excel.ods.[reader, writer]
- io.excel.xls.[reader, writer]
- io.excel.xlsb.[reader]
- io.excel.xlsm.[reader, writer]
- io.excel.xlsx.[reader, writer]
- io.hdf.[default_format, dropna_table]
- io.parquet.[engine]
- mode.[chained_assignment, sim_interactive, use_inf_a

In [3]:
# look at a few rows of the data
nfl_data.sample(6)

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,yrdln,yrdline100,ydstogo,ydsnet,GoalToGo,FirstDown,posteam,DefensiveTeam,desc,PlayAttempted,Yards.Gained,sp,Touchdown,ExPointResult,TwoPointConv,DefTwoPoint,Safety,Onsidekick,PuntResult,PlayType,Passer,Passer_ID,PassAttempt,PassOutcome,PassLength,AirYards,YardsAfterCatch,QBHit,PassLocation,InterceptionThrown,Interceptor,Rusher,Rusher_ID,RushAttempt,RunLocation,RunGap,Receiver,Receiver_ID,Reception,ReturnResult,Returner,BlockingPlayer,Tackler1,Tackler2,FieldGoalResult,FieldGoalDistance,Fumble,RecFumbTeam,RecFumbPlayer,Sack,Challenge.Replay,ChalReplayResult,Accepted.Penalty,PenalizedTeam,PenaltyType,PenalizedPlayer,Penalty.Yards,PosTeamScore,DefTeamScore,ScoreDiff,AbsScoreDiff,HomeTeam,AwayTeam,Timeout_Indicator,Timeout_Team,posteam_timeouts_pre,HomeTimeouts_Remaining_Pre,AwayTimeouts_Remaining_Pre,HomeTimeouts_Remaining_Post,AwayTimeouts_Remaining_Post,No_Score_Prob,Opp_Field_Goal_Prob,Opp_Safety_Prob,Opp_Touchdown_Prob,Field_Goal_Prob,Safety_Prob,Touchdown_Prob,ExPoint_Prob,TwoPoint_Prob,ExpPts,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season,Unnamed: 102,Unnamed: 103,Unnamed: 104
260946,2014-12-07,2014120703,8,2,3.0,06:36,7,2196.0,39.0,HOU,16.0,16.0,1,59,0.0,0.0,JAC,HOU,(6:36) (Shotgun) B.Bortles up the middle to HO...,1,0,0,0,NaN,NaN,NaN,0.0,0.0,NaN,Run,NaN,None,0,NaN,NaN,0,0,0,NaN,0,NaN,B.Bortles,00-0031407,1,middle,NaN,NaN,None,0,NaN,NaN,NaN,R.Pickett,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,0.0,7.0,10.0,-3.0,3.0,JAC,HOU,0,None,2,2,2,2,2,0.077531,0.025933,0.000059,0.039261,0.426578,0.001909,0.428728,0.0,0.0,3.931903,-1.233313,NaN,NaN,0.522683,0.477317,0.469796,0.530204,0.522683,-0.052888,NaN,NaN,2014.0,NaN,NaN,NaN
245818,2014-10-26,2014102609,6,2,3.0,08:47,9,2327.0,4.0,ARI,48.0,52.0,10,46,0.0,0.0,ARI,PHI,(8:47) (Shotgun) C.Palmer pass incomplete shor...,1,0,0,0,NaN,NaN,NaN,0.0,0.0,NaN,Pass,C.Palmer,00-0021429,1,Incomplete Pass,Short,-3,0,0,left,0,NaN,NaN,None,0,NaN,NaN,Jo. Brown,00-0031051,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,0.0,7.0,7.0,0.0,0.0,ARI,PHI,0,None,2,2,3,2,3,0.135482,0.113030,0.001225,0.171771,0.284993,0.004076,0.289424,0.0,0.0,1.345164,-1.367907,-1.656288,0.288381,0.512095,0.487905,0.470927,0.529073,0.512095,-0.041168,-0.049692,0.008524,2014.0,NaN,NaN,NaN
357009,2016-12-24,2016122400,15,3,1.0,07:56,8,1376.0,37.0,MIA,38.0,38.0,10,52,0.0,1.0,BUF,MIA,(7:56) (Shotgun) T.Taylor pass short right to ...,1,13,0,0,NaN,NaN,NaN,0.0,0.0,NaN,Pass,T.Taylor,00-0028118,1,Complete,Short,11,2,0,right,0,NaN,NaN,None,0,NaN,NaN,S.Watkins,00-0031325,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,0.0,7.0,21.0,-14.0,14.0,BUF,MIA,0,None,3,3,3,3,3,0.002963,0.059126,0.000172,0.090445,0.347072,0.003165,0.497057,0.0,0.0,3.716111,0.552529,0.478796,0.073733,0.159014,0.840986,0.170016,0.829984,0.159014,0.011002,0.010063,0.000938,2016.0,NaN,NaN,NaN
144286,2012-09-30,2012093001,5,1,1.0,00:59,1,2759.0,37.0,TEN,11.0,11.0,10,43,0.0,0.0,HOU,TEN,(:59) A.Foster right guard to TEN 4 for 7 yard...,1,7,0,0,NaN,NaN,NaN,0.0,0.0,NaN,Run,NaN,None,0,NaN,NaN,0,0,0,NaN,0,NaN,A.Foster,00-0026796,1,right,guard,NaN,None,0,NaN,NaN,NaN,J.Casey,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,0.0,7.0,0.0,7.0,7.0,HOU,TEN,0,None,2,2,3,2,3,0.006641,0.023349,0.000013,0.035018,0.386116,0.002353,0.546510,0.0,0.0,4.673429,0.000865,NaN,NaN,0.825310,0.174690,0.824368,0.175632,0.825310,-0.000942,NaN,NaN,2012.0,NaN,NaN,NaN
195490,2013-10-13,2013101309,8,2,1.0,08:41,9,2321.0,42.0,SEA,38.0,62.0,10,20,0.0,0.0,SEA,TEN,(8:41) R.Wilson pass short middle to M.Lynch t...,1,8,0,0,NaN,NaN,NaN,0.0,0.0,NaN,Pass,R.Wilson,00-0029263,1,Complete,Short,4,4,0,middle,0,NaN,NaN,None,0,NaN,NaN,M.Lynch,00-0025399,1,NaN,NaN,NaN,Z.Brown,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0,3.0,-3.0,3.0,SEA,TEN,0,None,3,3,3,3,3,0.149500,0.096184,0.001175,0.142947,0.243108,0.002916,0.364169,0.0,0.0,

In [4]:
# get the number of missing data points per column
missing_values_count = nfl_data.isna().sum()
print(missing_values_count[0:10])

Date                0
GameID              0
Drive               0
qtr                 0
down            54218
time              188
TimeUnder           0
TimeSecs          188
PlayTimeDiff      374
SideofField       450
dtype: int64


In [5]:
nfl_data.shape

(362447, 105)

In [6]:
# how many total missing values do we have?
total_missing = missing_values_count.sum()

# percent of data that is missing
num_total_cells = nfl_data.size
percent = total_missing / num_total_cells * 100
f"{percent:.2f} % is missing"

'26.95 % is missing'

In [8]:
# remove all columns with at least one missing value
col_names_no_nans = (missing_values_count == 0)
col_names_no_nans = col_names_no_nans[col_names_no_nans].index.tolist()

columns_with_na_dropped = nfl_data[col_names_no_nans]
columns_with_na_dropped.head()

,Date,GameID,Drive,qtr,TimeUnder,ydstogo,ydsnet,PlayAttempted,Yards.Gained,sp,Touchdown,QBHit,HomeTeam,AwayTeam,Timeout_Indicator,Timeout_Team,posteam_timeouts_pre,HomeTimeouts_Remaining_Pre,AwayTimeouts_Remaining_Pre,HomeTimeouts_Remaining_Post,AwayTimeouts_Remaining_Post,ExPoint_Prob,TwoPoint_Prob
0,2009-09-10,2009091000,1,1,15,0,0,1,39,0,0,0,PIT,TEN,0,None,3,3,3,3,3,0.0,0.0
1,2009-09-10,2009091000,1,1,15,10,5,1,5,0,0,0,PIT,TEN,0,None,3,3,3,3,3,0.0,0.0
2,2009-09-10,2009091000,1,1,15,5,2,1,-3,0,0,0,PIT,TEN,0,None,3,3,3,3,3,0.0,0.0
3,2009-09-10,2009091000,1,1,14,8,2,1,0,0,0,0,PIT,TEN,0,None,3,3,3,3,3,0.0,0.0
4,2009-09-10,2009091000,1,1,14,8,2,1,0,0,0,0,PIT,TEN,0,None,3,3,3,3,3,0.0,0.0


In [9]:
# just how much data did we lose? 
print("Columns in original dataset: %d \n" % len(nfl_data.columns))
print("Columns with na's dropped: %d" % len(columns_with_na_dropped.columns))

Columns in original dataset: 105 

Columns with na's dropped: 23


In [10]:
# replace all NA's with 0
filled = nfl_data.fillna(0)
filled.head()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,yrdln,yrdline100,ydstogo,ydsnet,GoalToGo,FirstDown,posteam,DefensiveTeam,desc,PlayAttempted,Yards.Gained,sp,Touchdown,ExPointResult,TwoPointConv,DefTwoPoint,Safety,Onsidekick,PuntResult,PlayType,Passer,Passer_ID,PassAttempt,PassOutcome,PassLength,AirYards,YardsAfterCatch,QBHit,PassLocation,InterceptionThrown,Interceptor,Rusher,Rusher_ID,RushAttempt,RunLocation,RunGap,Receiver,Receiver_ID,Reception,ReturnResult,Returner,BlockingPlayer,Tackler1,Tackler2,FieldGoalResult,FieldGoalDistance,Fumble,RecFumbTeam,RecFumbPlayer,Sack,Challenge.Replay,ChalReplayResult,Accepted.Penalty,PenalizedTeam,PenaltyType,PenalizedPlayer,Penalty.Yards,PosTeamScore,DefTeamScore,ScoreDiff,AbsScoreDiff,HomeTeam,AwayTeam,Timeout_Indicator,Timeout_Team,posteam_timeouts_pre,HomeTimeouts_Remaining_Pre,AwayTimeouts_Remaining_Pre,HomeTimeouts_Remaining_Post,AwayTimeouts_Remaining_Post,No_Score_Prob,Opp_Field_Goal_Prob,Opp_Safety_Prob,Opp_Touchdown_Prob,Field_Goal_Prob,Safety_Prob,Touchdown_Prob,ExPoint_Prob,TwoPoint_Prob,ExpPts,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season,Unnamed: 102,Unnamed: 103,Unnamed: 104
0,2009-09-10,2009091000,1,1,0.0,15:00,15,3600.0,0.0,TEN,30.0,30.0,0,0,0.0,0.0,PIT,TEN,R.Bironas kicks 67 yards from TEN 30 to PIT 3....,1,39,0,0,0,0,0,0.0,0.0,0,Kickoff,0,None,0,0,0,0,0,0,0,0,0,0,None,0,0,0,0,None,0,0,S.Logan,0,M.Griffin,0,0,0,0.0,0,0,0.0,0.0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,PIT,TEN,0,None,3,3,3,3,3,0.001506,0.179749,0.006639,0.281138,0.213700,0.003592,0.313676,0.0,0.0,0.323526,2.014474,0.000000,0.000000,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,0.000000,0.000000,2009.0,0.0,0.0,0.0
1,2009-09-10,2009091000,1,1,1.0,14:53,15,3593.0,7.0,PIT,42.0,58.0,10,5,0.0,0.0,PIT,TEN,(14:53) B.Roethlisberger pass short left to H....,1,5,0,0,0,0,0,0.0,0.0,0,Pass,B.Roethlisberger,00-0022924,1,Complete,Short,-3,8,0,left,0,0,0,None,0,0,0,H.Ward,00-0017162,1,0,0,0,C.Hope,0,0,0,0.0,0,0,0.0,0.0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,PIT,TEN,0,None,3,3,3,3,3,0.000969,0.108505,0.001061,0.169117,0.293700,0.003638,0.423011,0.0,0.0,2.338000,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009.0,0.0,0.0,0.0
2,2009-09-10,2009091000,1,1,2.0,14:16,15,3556.0,37.0,PIT,47.0,53.0,5,2,0.0,0.0,PIT,TEN,(14:16) W.Parker right end to PIT 44 for -3 ya...,1,-3,0,0,0,0,0,0.0,0.0,0,Run,0,None,0,0,0,0,0,0,0,0,0,W.Parker,00-0022250,1,right,end,0,None,0,0,0,0,S.Tulloch,0,0,0,0.0,0,0,0.0,0.0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,PIT,TEN,0,None,3,3,3,3,3,0.001057,0.105106,0.000981,0.162747,0.304805,0.003826,0.421478,0.0,0.0,2.415907,-1.402760,0.000000,0.000000,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.000000,0.000000,2009.0,0.0,0.0,0.0
3,2009-09-10,2009091000,1,1,3.0,13:35,14,3515.0,41.0,PIT,44.0,56.0,8,2,0.0,0.0,PIT,TEN,(13:35) (Shotgun) B.Roethlisberger pass incomp...,1,0,0,0,0,0,0,0.0,0.0,0,Pass,B.Roethlisberger,00-0022924,1,Incomplete Pass,Deep,34,0,0,right,0,0,0,None,0,0,0,M.Wallace,00-0026901,0,0,0,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,PIT,TEN,0,None,3,3,3,3,3,0.001434,0.149088,0.001944,0.234801,0.289336,0.004776,0.318621,0.0,0.0,1.013147,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009.0,0.0,0.0,0.0
4,2009-09-10,2009091000,1,1,4.0,13:27,14,3507.0,8.0,PIT,44.0,56.0,8,2,0.0,1.0,PIT,TEN,(13:27) (Punt formation) D.Sepulveda punts 54 ...,1,0,0,0,0,0,0,0.0,0.0,Clean,Punt,0,None,0,0,0,0,0,0,0,0,0,0,None,0,0,0,0,None,0,0,0,0,0,0,0,0,0.0,0,0,0.0,0.0,0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,PIT,TEN,0,None,3,3,3,3,3,0.001861,0.213480,0.003279,0.322262,0.244603,0.006404,0.208111,0.0,0.0,-0.699436,2.097796,0.000000,0.000000,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,0.000000,0.000000,2009.0,0.0,0.0,0.0


In [11]:
# replace all NA's the value that comes directly after it in the same column, 
# then replace all the reamining na's with 0
bfilled = nfl_data.fillna(method="bfill").fillna(0)
bfilled.head()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,yrdln,yrdline100,ydstogo,ydsnet,GoalToGo,FirstDown,posteam,DefensiveTeam,desc,PlayAttempted,Yards.Gained,sp,Touchdown,ExPointResult,TwoPointConv,DefTwoPoint,Safety,Onsidekick,PuntResult,PlayType,Passer,Passer_ID,PassAttempt,PassOutcome,PassLength,AirYards,YardsAfterCatch,QBHit,PassLocation,InterceptionThrown,Interceptor,Rusher,Rusher_ID,RushAttempt,RunLocation,RunGap,Receiver,Receiver_ID,Reception,ReturnResult,Returner,BlockingPlayer,Tackler1,Tackler2,FieldGoalResult,FieldGoalDistance,Fumble,RecFumbTeam,RecFumbPlayer,Sack,Challenge.Replay,ChalReplayResult,Accepted.Penalty,PenalizedTeam,PenaltyType,PenalizedPlayer,Penalty.Yards,PosTeamScore,DefTeamScore,ScoreDiff,AbsScoreDiff,HomeTeam,AwayTeam,Timeout_Indicator,Timeout_Team,posteam_timeouts_pre,HomeTimeouts_Remaining_Pre,AwayTimeouts_Remaining_Pre,HomeTimeouts_Remaining_Post,AwayTimeouts_Remaining_Post,No_Score_Prob,Opp_Field_Goal_Prob,Opp_Safety_Prob,Opp_Touchdown_Prob,Field_Goal_Prob,Safety_Prob,Touchdown_Prob,ExPoint_Prob,TwoPoint_Prob,ExpPts,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season,Unnamed: 102,Unnamed: 103,Unnamed: 104
0,2009-09-10,2009091000,1,1,1.0,15:00,15,3600.0,0.0,TEN,30.0,30.0,0,0,0.0,0.0,PIT,TEN,R.Bironas kicks 67 yards from TEN 30 to PIT 3....,1,39,0,0,0,Failure,0,0.0,0.0,Clean,Kickoff,B.Roethlisberger,None,0,Complete,Short,0,0,0,left,0,T.Polamalu,W.Parker,None,0,right,end,H.Ward,None,0,Fair Catch,S.Logan,J.Ringer,M.Griffin,J.Farrior,No Good,37,0.0,0,K.Fox,0.0,0.0,0,0.0,PIT,Illegal Formation,T.Polamalu,0.0,0.0,0.0,0.0,0.0,PIT,TEN,0,None,3,3,3,3,3,0.001506,0.179749,0.006639,0.281138,0.213700,0.003592,0.313676,0.0,0.0,0.323526,2.014474,-1.068169,1.146076,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,-0.032244,0.036899,2009.0,2009.0,2009.0,2012.0
1,2009-09-10,2009091000,1,1,1.0,14:53,15,3593.0,7.0,PIT,42.0,58.0,10,5,0.0,0.0,PIT,TEN,(14:53) B.Roethlisberger pass short left to H....,1,5,0,0,0,Failure,0,0.0,0.0,Clean,Pass,B.Roethlisberger,00-0022924,1,Complete,Short,-3,8,0,left,0,T.Polamalu,W.Parker,None,0,right,end,H.Ward,00-0017162,1,Fair Catch,S.Logan,J.Ringer,C.Hope,J.Farrior,No Good,37,0.0,0,K.Fox,0.0,0.0,0,0.0,PIT,Illegal Formation,T.Polamalu,0.0,0.0,0.0,0.0,0.0,PIT,TEN,0,None,3,3,3,3,3,0.000969,0.108505,0.001061,0.169117,0.293700,0.003638,0.423011,0.0,0.0,2.338000,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009.0,2009.0,2009.0,2012.0
2,2009-09-10,2009091000,1,1,2.0,14:16,15,3556.0,37.0,PIT,47.0,53.0,5,2,0.0,0.0,PIT,TEN,(14:16) W.Parker right end to PIT 44 for -3 ya...,1,-3,0,0,0,Failure,0,0.0,0.0,Clean,Run,B.Roethlisberger,None,0,Incomplete Pass,Deep,0,0,0,right,0,T.Polamalu,W.Parker,00-0022250,1,right,end,M.Wallace,None,0,Fair Catch,S.Logan,J.Ringer,S.Tulloch,J.Farrior,No Good,37,0.0,0,K.Fox,0.0,0.0,0,0.0,PIT,Illegal Formation,T.Polamalu,0.0,0.0,0.0,0.0,0.0,PIT,TEN,0,None,3,3,3,3,3,0.001057,0.105106,0.000981,0.162747,0.304805,0.003826,0.421478,0.0,0.0,2.415907,-1.402760,3.318841,-5.031425,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.106663,-0.156239,2009.0,2009.0,2009.0,2012.0
3,2009-09-10,2009091000,1,1,3.0,13:35,14,3515.0,41.0,PIT,44.0,56.0,8,2,0.0,0.0,PIT,TEN,(13:35) (Shotgun) B.Roethlisberger pass incomp...,1,0,0,0,0,Failure,0,0.0,0.0,Clean,Pass,B.Roethlisberger,00-0022924,1,Incomplete Pass,Deep,34,0,0,right,0,T.Polamalu,C.Johnson,None,0,middle,end,M.Wallace,00-0026901,0,Fair Catch,S.Logan,J.Ringer,B.Keisel,J.Farrior,No Good,37,0.0,0,K.Fox,0.0,0.0,0,0.0,PIT,Illegal Formation,T.Polamalu,0.0,0.0,0.0,0.0,0.0,PIT,TEN,0,None,3,3,3,3,3,0.001434,0.149088,0.001944,0.234801,0.289336,0.004776,0.318621,0.0,0.0,1.013147,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009.0,2009.0,2009.0,2012.0
4,2009-09-10,2009091000,1,1,4.0,13:27,14,3507.0,8.0,PIT,44.0,56.0,8,2,0.0,1.0,PIT,TEN,(13:27) (Punt formation) 

In [12]:
# Create indicator column for missing values in data nlf_data for the column 'down'
nfl_data["down_nan_indicator"] = np.where(nfl_data["down"].isna(), 1, 0 )

In [14]:
# print columns that have more than 50 missing values in data nlf_data
col_names_50_more_missing = (missing_values_count > 50)

col_names_50_more_missing[col_names_50_more_missing].index.tolist()


['down',
 'time',
 'TimeSecs',
 'PlayTimeDiff',
 'SideofField',
 'yrdln',
 'yrdline100',
 'GoalToGo',
 'FirstDown',
 'posteam',
 'DefensiveTeam',
 'ExPointResult',
 'TwoPointConv',
 'DefTwoPoint',
 'Safety',
 'PuntResult',
 'PlayType',
 'Passer',
 'Passer_ID',
 'PassOutcome',
 'PassLength',
 'AirYards',
 'PassLocation',
 'InterceptionThrown',
 'Interceptor',
 'Rusher',
 'Rusher_ID',
 'RunLocation',
 'RunGap',
 'Receiver',
 'Receiver_ID',
 'ReturnResult',
 'Returner',
 'BlockingPlayer',
 'Tackler1',
 'Tackler2',
 'FieldGoalResult',
 'FieldGoalDistance',
 'Fumble',
 'RecFumbTeam',
 'RecFumbPlayer',
 'Sack',
 'ChalReplayResult',
 'Accepted.Penalty',
 'PenalizedTeam',
 'PenaltyType',
 'PenalizedPlayer',
 'Penalty.Yards',
 'PosTeamScore',
 'DefTeamScore',
 'ScoreDiff',
 'AbsScoreDiff',
 'No_Score_Prob',
 'Opp_Field_Goal_Prob',
 'Opp_Safety_Prob',
 'Opp_Touchdown_Prob',
 'Field_Goal_Prob',
 'Safety_Prob',
 'Touchdown_Prob',
 'ExpPts',
 'EPA',
 'airEPA',
 'yacEPA',
 'Home_WP_pre',
 'Away_WP_p

In [15]:
# What are the data types for each column?
nfl_data.dtypes

Date                   object
GameID                  int64
Drive                   int64
qtr                     int64
down                  float64
                       ...   
Season                float64
Unnamed: 102          float64
Unnamed: 103          float64
Unnamed: 104          float64
down_nan_indicator      int64
Length: 106, dtype: object

In [16]:
# What are the column names in the data?
nfl_data.columns.tolist()

['Date',
 'GameID',
 'Drive',
 'qtr',
 'down',
 'time',
 'TimeUnder',
 'TimeSecs',
 'PlayTimeDiff',
 'SideofField',
 'yrdln',
 'yrdline100',
 'ydstogo',
 'ydsnet',
 'GoalToGo',
 'FirstDown',
 'posteam',
 'DefensiveTeam',
 'desc',
 'PlayAttempted',
 'Yards.Gained',
 'sp',
 'Touchdown',
 'ExPointResult',
 'TwoPointConv',
 'DefTwoPoint',
 'Safety',
 'Onsidekick',
 'PuntResult',
 'PlayType',
 'Passer',
 'Passer_ID',
 'PassAttempt',
 'PassOutcome',
 'PassLength',
 'AirYards',
 'YardsAfterCatch',
 'QBHit',
 'PassLocation',
 'InterceptionThrown',
 'Interceptor',
 'Rusher',
 'Rusher_ID',
 'RushAttempt',
 'RunLocation',
 'RunGap',
 'Receiver',
 'Receiver_ID',
 'Reception',
 'ReturnResult',
 'Returner',
 'BlockingPlayer',
 'Tackler1',
 'Tackler2',
 'FieldGoalResult',
 'FieldGoalDistance',
 'Fumble',
 'RecFumbTeam',
 'RecFumbPlayer',
 'Sack',
 'Challenge.Replay',
 'ChalReplayResult',
 'Accepted.Penalty',
 'PenalizedTeam',
 'PenaltyType',
 'PenalizedPlayer',
 'Penalty.Yards',
 'PosTeamScore',
 'De

In [17]:
def _prepare_data(data, **kwargs):
    data = data.select_dtypes(exclude=["object"])
    train, test = train_test_split(data, **kwargs)

    return train, test

def _separate_target(data, target="Drive"):
    X_train = data[[i for i in data.columns if i != target]]  # all columns except target
    y_train = data[target]

    return X_train, y_train

In [18]:
# Split the data into test and train

# we do not consider here any id columns or something like that otherwise it would become not exercise but project.
# and we do not take into account about data leakage too, which happens here :)
train, test = _prepare_data(bfilled, test_size=0.3, random_state=42)
X_bfill_train, y_bfill_train = _separate_target(train, target="Drive")
X_bfill_test, y_bfill_test = _separate_target(train, target="Drive")

In [19]:
# How many rows in the train set?
len(train)

253712

In [20]:
# Import RandomForest model and fit to the nld_data data to predict the column PlayType. 
# You can choose another column if you want.
from sklearn.ensemble import RandomForestRegressor

rfc = RandomForestRegressor(random_state=42, verbose=1)


# could not train on nan data :)
rfc.fit(X_bfill_train, y_bfill_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 12.5min finished


RandomForestRegressor(random_state=42, verbose=1)

In [21]:
# Predict 
y_bfill_pred = rfc.predict(X_bfill_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   12.9s finished


In [22]:
# Evaluate
from sklearn.metrics import r2_score

r2_score_bfill = r2_score(y_bfill_pred, y_bfill_test)

In [23]:
# Retrain the model on datasets filled and bfilled
train_fill, test_fill = _prepare_data(filled, test_size=0.3, random_state=42)
X_fill_train, y_fill_train = _separate_target(train_fill, target="Drive")
X_fill_test, y_fill_test = _separate_target(test_fill, target="Drive")

In [24]:
# Train, predict and evaluate
rfc.fit(X_fill_train, y_fill_train)

y_fill_pred = rfc.predict(X_fill_test)
r2_score_fill = r2_score(y_fill_pred, y_fill_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 10.7min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    6.4s finished


In [25]:
# Compare results
r2_score_fill - r2_score_bfill

-0.07604355044507016